In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

## Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result_w = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_w.columns=['user_id', 'actual']
result_w.head(3)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [5]:
items = data_train.item_id.unique()
items

array([ 1004906,  1033142,  1036325, ..., 15722756, 17170636, 15716393])

In [6]:
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights.columns = ['item_id', 'sales_value']
items_weights['weight'] = items_weights['sales_value']/items_weights['sales_value'].sum()
items_weights = items_weights.drop(columns = 'sales_value', axis = 1)
items_weights

,item_id,weight
0,25671,2.969296e-06
1,26081,1.403822e-07
2,26093,2.254623e-07
3,26190,2.183723e-07
4,26355,2.807644e-07
...,...,...
86860,17381856,0.000000e+00
86861,17382205,1.132983e-06
86862,17383227,6.366828e-07
86863,17827644,3.545005e-07


In [7]:
def weighted_random_recommendation(items, items_weights, n=5):

    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_weights['item_id'])
    items_weights = np.array(items_weights['weight'])

    recs = np.random.choice(items, n, p= items_weights, replace=False)

    return recs.tolist()

In [8]:
result_w['weighted_random_recommendation'] = result_w['user_id'].apply(lambda x: weighted_random_recommendation(items, items_weights, n=5))
result_w.head(3)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[977927, 979022, 1058997, 1058214, 9221647]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[970030, 834993, 1128016, 833238, 6534178]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1933525, 1389035, 1081467, 1010847, 6534178]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [9]:
result = pd.read_csv('predictions_basic.csv')
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]"


In [10]:
#преобразуем тип данных

result['actual'] = result['actual'].apply(lambda x: [int(el) for el in x.replace('[', '').replace(']', '').split()])

col_name = result.columns.values.tolist()[2:]
for i in col_name:
  result[i] = result[i].apply(lambda x: [int(el) for el in x.strip('[]').replace(' ', '').split(',')])

In [11]:
result['weighted_random_recommendation'] = result_w['weighted_random_recommendation']

In [12]:
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[977927, 979022, 1058997, 1058214, 9221647]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[970030, 834993, 1128016, 833238, 6534178]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]","[1933525, 1389035, 1081467, 1010847, 6534178]"


In [13]:
def precision_at_k(actual_list, recommended_list, k=5):
    actual_list = np.array(actual_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(actual_list, recommended_list)
    precision = flags.sum() / k

    return precision


In [14]:
# рассчитаем метрику
col_name = result.columns.values.tolist()[2:]
tab_metrics = {}
for i in col_name:
  metric = result.apply(lambda x: precision_at_k(x['actual'], x[i], k=5), axis = 1).mean()
  tab_metrics[i] = metric


In [15]:
tab_metrics

{'random_recommendation': 0.0005876591576885408,
 'popular_recommendation': 0.15523996082272282,
 'itemitem': 0.03359451518119491,
 'cosine': 0.03525954946131244,
 'tfidf': 0.036141038197845254,
 'own_purchases': 0.17992164544564151,
 'weighted_random_recommendation': 0.02272282076395691}

#### вывод: из данных алгоритмов лучшее качество показывает алгоритм рекомендаций на основе собственных покупок

## Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [16]:
popularity_data_train = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity_data_train.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity_data_train.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [17]:
top_5000 = popularity_data_train.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [18]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
# sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
user_item_matrix.shape

(2499, 86865)

In [20]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

0.5804511409122447

#### Для обучения модли мы должны на вход подать разреженную матрицу sparse_user_item, где строки user а столбцы item. Здесь важен порядок.

#### Переведем user_item_matrix в формат saprse matrix

In [21]:
%%time
# переведем user_item_matrix в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

CPU times: user 4.04 s, sys: 627 µs, total: 4.04 s
Wall time: 4.02 s


### Построим матрицу user_item_matrix_test  ДЛЯ data_test где строки user а столбцы item. Здесь важен порядок! Заполним ячейки 1 или 0, в зависимости от их содержимого. Она будет нуждна для расчета метрик

In [22]:
%%time

# Заведем фиктивный item_id (если юзер покупал товары НЕ из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix_test = pd.pivot_table(data_test,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix_test[user_item_matrix_test > 0] = 1 # так как в итоге хотим предсказать
user_item_matrix_test = user_item_matrix_test.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
# sparse_user_item_test = csr_matrix(user_item_matrix).tocsr()

user_item_matrix_test.head(3)

CPU times: user 13.9 s, sys: 12.1 ms, total: 14 s
Wall time: 13.9 s


item_id,29512,30356,32392,32439,34873,40405,42852,43094,44049,44522,...,17959192,17959243,17959460,17974316,17974317,17991689,17991691,18000012,18024155,18024556
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
user_item_matrix_test.shape

(2042, 24329)

In [24]:
%%time
sparse_user_item_test = csr_matrix(user_item_matrix).tocsr()

CPU times: user 4.59 s, sys: 655 µs, total: 4.59 s
Wall time: 4.65 s


In [25]:
# Подготавливает вспомогательные словари

# перенумеруем пользователей и товары,
# чтобы они задавались числами от 0 до номера количества товаров.

# userids array содержит индексы user_id, какие-то значения могут быть пропущены
userids = user_item_matrix.index.values

# itemids array содержит индексы item_id, какие-то значения могут быть пропущены
itemids = user_item_matrix.columns.values

# Формируем матрицы с последовательными числами, т.е. без пропусков
# номеров, перенумеруем для задания последовательных номеров userids, itemids
# числами от 0 до фактического количества userids, itemids

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))


# Формируем словари где ключ это перенумерованный порядковый номер userids, itemids
# значение ключа это фактическое значение из данных userids, itemids
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# Другими словами, стовим фактическому номеру его порядковое значение
#  от 0 до n, подставляем в модель и снова переходим от порядковог номера к фактическому

## Создаем и обучаем модель ItemItemRecommender

In [26]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 64.8 MB/s eta 0:00:00


In [27]:
from scipy.sparse import csr_matrix, coo_matrix
import implicit
# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
#from implicit.evaluation import train_test_split
#from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [28]:
%%time

model_ItemItem = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

CPU times: user 11 µs, sys: 2 µs, total: 13 µs
Wall time: 17.4 µs


In [29]:
%%time

# model.fit(csr_matrix(user_item_matrix).tocsr(), show_progress=True)
model_ItemItem.fit(sparse_user_item, show_progress=True)

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 15.4 s, sys: 94.4 ms, total: 15.5 s
Wall time: 14.8 s


##### Сделаем рекомендацию для userid[2], т.е. по его индексу а не фактическому значению его user_id

In [30]:
%%time
user_n = 0
id_n, scores_n = model_ItemItem.recommend(userid=userids[user_n],
                        user_items=sparse_user_item[user_n],
                        N=5,
                        filter_already_liked_items=False,
                        filter_items=None,
                        recalculate_user=True)

CPU times: user 666 µs, sys: 933 µs, total: 1.6 ms
Wall time: 3.5 ms


In [31]:
id_n, scores_n

(array([34707, 23590, 39619, 25064,  7933], dtype=int32),
 array([65352., 50502., 23817., 18957., 15833.]))

#### id_n содержит индексы user_item_matrix / sparse_user_item, для перехода к item_id:

In [32]:
res_n = [id_to_itemid[rec] for rec in id_n]
res_n

[1082185, 981760, 1127831, 995242, 840361]

##### Сделаем рекомендацию для для всех user

In [33]:
%%time
# userids array содержит индексы user_id,
# какие-то значения могут быть пропущены
# userids = user_item_matrix.index.values


ids, scores = model_ItemItem.recommend(userid=userid_to_id,
                        user_items=sparse_user_item,
                        N=5,
                        filter_already_liked_items=False,
                        filter_items=None,
                        recalculate_user=True)

CPU times: user 707 ms, sys: 4.42 ms, total: 712 ms
Wall time: 1.23 s


##### Занесем рекомендованные данные для всех пользователей в итоговый результирующий датафрейм

Создадим вспомогательный столбец 'userid_to_id' для записи
индексов, соответствующих индексации в матрице sparse_user_item.
Затем генератором переводим порядковые номера рекомендаций, соответствующих индексации в матрице sparse_user_item, в их фактические значения.


Замечание: вместо стобца 'user_id' можно использовать индексы pd.DataFrame, возможно не во всех случаях.

In [34]:
%%time
result['userid_to_id'] = result['user_id'].apply(
    lambda x: userid_to_id[x] )

result['itemitem'] = result['userid_to_id'].apply(
    lambda x: [id_to_itemid[rec] for rec in ids[x]])

result[['user_id', 'actual', 'itemitem']].head(5)

CPU times: user 26.8 ms, sys: 920 µs, total: 27.7 ms
Wall time: 27.8 ms


,user_id,actual,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 981760, 1127831, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1082185, 981760, 1098066, 826249, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 981760, 1127831, 995242, 1098066]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 981760, 1127831, 995242, 1098066]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1082185, 981760, 1098066, 995242, 1127831]"


In [35]:
result.apply(lambda x: precision_at_k(x['actual'], x['itemitem'], k=5), axis = 1).mean()

0.15406464250734575

### вывод: при изменении параметра (топ-5000) алгоритма определения подобия товара посредством анализа данных о предпочтениях пользователя метрика улучшилась с 0,034 до 0,154,  однако не дотянула до результата алгоритма рекомендаций на основе собственных покупок